# Suggestion Writer in Python

This code is main edited by Enze Wang

Fixed by Yicen Liu and Hua Tong

# Attention

Please put four .csv file in data folder and put this code in main folder instead of code folder.

This code want to write suggesion for business from the analysis above.  



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import itertools
from collections import Counter
from nltk import FreqDist
from collections import Counter
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize, PorterStemmer, pos_tag 
from nltk.corpus import stopwords
from nltk.text import TextCollection
from nltk.corpus import wordnet
import nltk.tokenize as nt
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

[nltk_data] Downloading package stopwords to /Users/enze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/enze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/enze/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
Noun_results=pd.read_csv("data/Results_Analysis_Noun_Class.csv")
Review_results=pd.read_csv("data/Results_Analysis_Noun_Attitude.csv")
pizza_review=pd.read_csv("data/pizza_review.csv")
pizza_business=pd.read_csv("data/pizza_business.csv")
pizza_business_clean_review=pd.read_csv("data/pizza_business_clean_review.csv")
Word_Review_Matrix=pd.read_csv("data/Word_Review_Matrix.csv")

# Business Suggestion

Suggestion for business is simple. For example, if one business offer Alcohol, we can suggest them to keep. If not, suggest them to offer alcohol as possible.  

Therefore in our suggestion here, we use 1 as the business rating is high option and 0 as the low option. The detailed suggestion will raise by Shiny app code. 

For np.nan in the value, we default give low option rating suggestion. 

Some special category, such as Delivery, we will mention and give warning to those given delivery to raise their delivery quality and service instead of cancel delivery.

In [3]:
pizza_business["Alcohol"]

0                None
1                 NaN
2                None
3       beer_and_wine
4       beer_and_wine
            ...      
1555             None
1556             None
1557             None
1558             None
1559         full_bar
Name: Alcohol, Length: 1560, dtype: object

In [4]:
set(pizza_business["Alcohol"])

{'None', 'beer_and_wine', 'full_bar', nan}

In [5]:
Alcohol_Suggestion=pizza_business["Alcohol"]
Alcohol_Suggestion=Alcohol_Suggestion.replace(['beer_and_wine','full_bar'],"1")
Alcohol_Suggestion=Alcohol_Suggestion.replace(['None',np.nan],"0")

In [6]:
Alcohol_Suggestion

0       0
1       0
2       0
3       1
4       1
       ..
1555    0
1556    0
1557    0
1558    0
1559    1
Name: Alcohol, Length: 1560, dtype: object

In [7]:
pizza_business["BikeParking"]

0        True
1         NaN
2        True
3         NaN
4        True
        ...  
1555      NaN
1556    False
1557    False
1558     True
1559     True
Name: BikeParking, Length: 1560, dtype: object

In [8]:
set(pizza_business["BikeParking"])

{nan, False, True}

In [9]:
BikeParking_Suggestion=pizza_business["BikeParking"]
BikeParking_Suggestion=BikeParking_Suggestion.replace([True],"1")
BikeParking_Suggestion=BikeParking_Suggestion.replace([False,np.nan],"0")
BikeParking_Suggestion

0       1
1       0
2       1
3       0
4       1
       ..
1555    0
1556    0
1557    0
1558    1
1559    1
Name: BikeParking, Length: 1560, dtype: object

In [10]:
pizza_business["GoodForKids"]

0       True
1       True
2       True
3       True
4       True
        ... 
1555    True
1556    True
1557    True
1558    True
1559    True
Name: GoodForKids, Length: 1560, dtype: object

In [11]:
set(pizza_business["GoodForKids"])

{nan, False, True}

In [12]:
GoodForKids_Suggestion=pizza_business["GoodForKids"]
GoodForKids_Suggestion=GoodForKids_Suggestion.replace([True],"1")
GoodForKids_Suggestion=GoodForKids_Suggestion.replace([False,np.nan],"0")
GoodForKids_Suggestion

0       1
1       1
2       1
3       1
4       1
       ..
1555    1
1556    1
1557    1
1558    1
1559    1
Name: GoodForKids, Length: 1560, dtype: object

In [13]:
set(GoodForKids_Suggestion)

{'0', '1'}

In [14]:
pizza_business["NoiseLevel"]

0           quiet
1           quiet
2           quiet
3           quiet
4         average
          ...    
1555          NaN
1556      average
1557          NaN
1558      average
1559    very_loud
Name: NoiseLevel, Length: 1560, dtype: object

In [15]:
set(pizza_business["NoiseLevel"])

{'average', 'loud', nan, 'quiet', 'very_loud'}

In [16]:
NoiseLevel_Suggestion=pizza_business["NoiseLevel"]
NoiseLevel_Suggestion=NoiseLevel_Suggestion.replace(['average','quiet'],"1")
NoiseLevel_Suggestion=NoiseLevel_Suggestion.replace(['very_loud',np.nan,'loud'],"0")
NoiseLevel_Suggestion

0       1
1       1
2       1
3       1
4       1
       ..
1555    0
1556    1
1557    0
1558    1
1559    0
Name: NoiseLevel, Length: 1560, dtype: object

In [17]:
set(NoiseLevel_Suggestion)

{'0', '1'}

In [18]:
pizza_business["RestaurantsGoodForGroups"]

0        True
1       False
2        True
3        True
4        True
        ...  
1555     True
1556     True
1557     True
1558     True
1559     True
Name: RestaurantsGoodForGroups, Length: 1560, dtype: object

In [19]:
set(pizza_business["RestaurantsGoodForGroups"])

{False, True, nan}

In [20]:
RestaurantsGoodForGroups_Suggestion=pizza_business["RestaurantsGoodForGroups"]
RestaurantsGoodForGroups_Suggestion=RestaurantsGoodForGroups_Suggestion.replace([True],"1")
RestaurantsGoodForGroups_Suggestion=RestaurantsGoodForGroups_Suggestion.replace([False,np.nan],"0")
RestaurantsGoodForGroups_Suggestion

0       1
1       0
2       1
3       1
4       1
       ..
1555    1
1556    1
1557    1
1558    1
1559    1
Name: RestaurantsGoodForGroups, Length: 1560, dtype: object

In [21]:
set(RestaurantsGoodForGroups_Suggestion)

{'0', '1'}

In [22]:
pizza_business["RestaurantsReservations"]

0        True
1       False
2       False
3       False
4       False
        ...  
1555    False
1556    False
1557    False
1558    False
1559    False
Name: RestaurantsReservations, Length: 1560, dtype: object

In [23]:
set(pizza_business["RestaurantsReservations"])

{False, True, nan}

In [24]:
RestaurantsReservations_Suggestion=pizza_business["RestaurantsReservations"]
RestaurantsReservations_Suggestion=RestaurantsReservations_Suggestion.replace([True],"1")
RestaurantsReservations_Suggestion=RestaurantsReservations_Suggestion.replace([False,np.nan],"0")
RestaurantsReservations_Suggestion

0       1
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    0
1558    0
1559    0
Name: RestaurantsReservations, Length: 1560, dtype: object

In [25]:
set(RestaurantsReservations_Suggestion)

{'0', '1'}

In [26]:
pizza_business["WiFi"]

0         no
1         no
2         no
3         no
4       free
        ... 
1555      no
1556      no
1557    free
1558      no
1559      no
Name: WiFi, Length: 1560, dtype: object

In [27]:
set(pizza_business["WiFi"])

{'free', nan, 'no', 'paid'}

In [28]:
WiFi_Suggestion=pizza_business["WiFi"]
WiFi_Suggestion=WiFi_Suggestion.replace(['free'],"1")
WiFi_Suggestion=WiFi_Suggestion.replace(['no','paid',np.nan],"0")
WiFi_Suggestion

0       0
1       0
2       0
3       0
4       1
       ..
1555    0
1556    0
1557    1
1558    0
1559    0
Name: WiFi, Length: 1560, dtype: object

In [29]:
set(WiFi_Suggestion)

{'0', '1'}

In [30]:
pizza_business["RestaurantsDelivery"]

0        True
1        True
2        True
3        True
4        True
        ...  
1555      NaN
1556     True
1557    False
1558     True
1559     True
Name: RestaurantsDelivery, Length: 1560, dtype: object

In [31]:
set(pizza_business["RestaurantsDelivery"])

{False, True, nan}

In [32]:
RestaurantsDelivery_Suggestion=pizza_business["RestaurantsDelivery"]
RestaurantsDelivery_Suggestion=RestaurantsDelivery_Suggestion.replace([False],"1")
RestaurantsDelivery_Suggestion=RestaurantsDelivery_Suggestion.replace([True,np.nan],"0")
RestaurantsDelivery_Suggestion

0       0
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    1
1558    0
1559    0
Name: RestaurantsDelivery, Length: 1560, dtype: object

In [33]:
set(RestaurantsDelivery_Suggestion)

{'0', '1'}

In [34]:
pizza_business["BusinessParking"]

0          NaN
1          NaN
2          NaN
3          lot
4          lot
         ...  
1555       NaN
1556       NaN
1557       NaN
1558    street
1559       NaN
Name: BusinessParking, Length: 1560, dtype: object

In [35]:
Park=list(set(pizza_business["BusinessParking"]))
Park=[x for x in Park if str(x) != 'nan']
Park

['lot street',
 'lot validated',
 'garage',
 'street',
 'garage street',
 'garage validated street',
 'valet street',
 'valet garage validated street',
 'validated',
 'garage lot street',
 'valet validated',
 'valet',
 'valet lot',
 'lot',
 'valet garage street']

In [36]:
BusinessParking_Suggestion=pizza_business["BusinessParking"]
BusinessParking_Suggestion=BusinessParking_Suggestion.replace(Park,"1")
BusinessParking_Suggestion=BusinessParking_Suggestion.replace([np.nan],"0")
BusinessParking_Suggestion

0       0
1       0
2       0
3       1
4       1
       ..
1555    0
1556    0
1557    0
1558    1
1559    0
Name: BusinessParking, Length: 1560, dtype: object

In [37]:
set(BusinessParking_Suggestion)

{'0', '1'}

In [38]:
Business_Suggestion=pd.DataFrame([pizza_business["business_id"],
                                  Alcohol_Suggestion,
BikeParking_Suggestion,
GoodForKids_Suggestion,
NoiseLevel_Suggestion,
RestaurantsGoodForGroups_Suggestion,
RestaurantsReservations_Suggestion,
WiFi_Suggestion,
RestaurantsDelivery_Suggestion,
BusinessParking_Suggestion])
Business_Suggestion=Business_Suggestion.transpose()
Business_Suggestion

,business_id,Alcohol,BikeParking,GoodForKids,NoiseLevel,RestaurantsGoodForGroups,RestaurantsReservations,WiFi,RestaurantsDelivery,BusinessParking
0,ZkzutF0P_u0C0yTulwaHkA,0,1,1,1,1,1,0,0,0
1,39lLJK_rrYY2NYomSsQdUA,0,0,1,1,0,0,0,0,0
2,0y6alZmSLnPzmG5_kP5Quw,0,1,1,1,1,0,0,0,0
3,_Kp1IPTi17wBywYOd30raA,1,0,1,1,1,0,0,0,1
4,crBw_Hq3gCkppirgjnIl4A,1,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,0,0,1,0,1,0,0,0,0
1556,ZZGs9idRiKRHue2RyXw9uA,0,0,1,1,1,0,0,0,0
1557,v8GgrqLTs_NZZl2tkyvOgA,0,0,1,0,1,0,1,1,0
1558,qWcaGio9JsqrvgmQiIRtIQ,0,1,1,1,1,0,0,0,1


Check the number of 1 according to with 8_Analysis_Business

In [39]:
Counter(Business_Suggestion["Alcohol"])

Counter({'0': 1085, '1': 475})

In [40]:
Counter(Business_Suggestion["BikeParking"])

Counter({'1': 688, '0': 872})

In [41]:
Counter(Business_Suggestion["GoodForKids"])

Counter({'1': 1094, '0': 466})

In [42]:
Counter(Business_Suggestion["NoiseLevel"])

Counter({'1': 861, '0': 699})

In [43]:
Counter(Business_Suggestion["RestaurantsDelivery"])

Counter({'0': 1209, '1': 351})

In [44]:
Counter(Business_Suggestion["RestaurantsGoodForGroups"])

Counter({'1': 964, '0': 596})

In [45]:
Counter(Business_Suggestion["RestaurantsReservations"])

Counter({'1': 289, '0': 1271})

In [46]:
Counter(Business_Suggestion["WiFi"])

Counter({'0': 1131, '1': 429})

In [47]:
Counter(Business_Suggestion["BusinessParking"])

Counter({'0': 823, '1': 737})

Check, the business_id 0y6alZmSLnPzmG5_kP5Quw

In [48]:
Business_Suggestion.iloc[2]

business_id                 0y6alZmSLnPzmG5_kP5Quw
Alcohol                                          0
BikeParking                                      1
GoodForKids                                      1
NoiseLevel                                       1
RestaurantsGoodForGroups                         1
RestaurantsReservations                          0
WiFi                                             0
RestaurantsDelivery                              0
BusinessParking                                  0
Name: 2, dtype: object

In pizza_business:

In [49]:
pizza_business.iloc[2]

Unnamed: 0                                              2
business_id                        0y6alZmSLnPzmG5_kP5Quw
name                                          J J's Pizza
address                                   20542 Lorain Rd
city                                            Cleveland
latitude                                          41.4483
longitude                                        -81.8476
stars                                                 4.5
review_count                                           21
categories                    Pizza, Italian, Restaurants
Alcohol                                              None
Ambience                                              NaN
BikeParking                                          True
BusinessAcceptsCreditCards                           True
Caters                                               True
GoodForKids                                          True
HasTV                                                True
NoiseLevel    

No problem

# Noun suggestion

The noun suggestion suggest business to offer or add XXX service to their business, something like food or service. We will raise suggestion like:

Review contains XXX, XXX and XXX individually always have high review star rating compared with that without. Think about add word related service or food to increase your star rating. 

or

Review contains XXX, XXX and XXX individually always have low review star rating compared with that without. Think about remove or fix word related service or food to increase your star rating. 

The Noun suggestion is the most difficult part in our suggestion, because:

1. Some review contains "cheese" for example may something like "I want you offer cheese pizza". Therefore it is strange to provide suggestion like "Keep cheese in your food list" if this business review does not contain "cheese" but it does offer cheese.

Our suggestion will be divided into four part. Add, Keep, Fix, Avoid. 

Add means that you do not have positive noun and we suggest you add. 

Keep means that you do have positive noun and we suggest you keep. 

Fix means that you do have negative noun and we suggest you fix. 

Avoid means that you do not have negative noun and we suggest you avoid. 

In [50]:
Noun_results=Noun_results.rename(columns={"Unnamed: 0": "Index"})
Noun_results

,Index,Food,Lasso_Coefficients
0,1,(Intercept),3.648472
1,2,cheese,-0.017139
2,5,beer,0.038242
3,8,wine,0.067184
4,9,sausage,0.034191
5,15,bacon,0.068713
6,16,mozzarella,0.107084
7,18,shrimp,-0.041437
8,19,fish,-0.033910
9,21,spaghetti,-0.026370


In [51]:
New_Noun_results=Noun_results.drop([0])
New_Noun_results

,Index,Food,Lasso_Coefficients
1,2,cheese,-0.017139
2,5,beer,0.038242
3,8,wine,0.067184
4,9,sausage,0.034191
5,15,bacon,0.068713
6,16,mozzarella,0.107084
7,18,shrimp,-0.041437
8,19,fish,-0.033910
9,21,spaghetti,-0.026370
10,22,lettuce,-0.527233


In [52]:
Positive_Noun=New_Noun_results[New_Noun_results["Lasso_Coefficients"]>0]["Food"].tolist()
Negative_Noun=New_Noun_results[New_Noun_results["Lasso_Coefficients"]<0]["Food"].tolist()
Positive_Noun

['beer',
 'wine',
 'sausage',
 'bacon',
 'mozzarella',
 'bbq',
 'gelato',
 'cocktail',
 'pesto',
 'margherita',
 'chocolate',
 'crunchy',
 'arugula',
 'artichoke',
 'margarita',
 'prosciutto',
 'tiramisu',
 'cider',
 'donut',
 'cannoli',
 'cauliflower',
 'hummus',
 'cooky',
 'carbonara',
 'vinaigrette',
 'salami',
 'jam',
 'pistachio',
 'sangria',
 'espresso',
 'dumpling',
 'bourbon']

In [53]:
pizza_business_clean_review

,Unnamed: 0,review_id,clean_review
0,0,hfXVxrlD6fr1kXfz6sR13Q,parking tough get get go much also get little ...
1,1,EDQpLLFiAxeSxWI5ipwzBQ,server extensive knowledge wine various menu s...
2,2,RWmT90rP7wYo57dG1cyvww,fast forward every trip nyc pitt includes visi...
3,3,0jduztItUhOh3FzKGjMxZQ,far favorite cleveland/akron restaurant great ...
4,4,sxfmzhMvZ8LlI1x4fhNjKg,food good usually get italian sub pizza would ...
...,...,...,...
67151,67151,gcNRaJMDlSVs8YlqwpsRuw,originally side ohio always really enjoyed jet...
67152,67152,yFxDOdiDb8HPaK1io81kiw,yes yes yes loved going place shadyside ecstat...
67153,67153,Z_xmoLgIMB4As90t0a5eBQ,went lunch meeting amidst typical suburban str...
67154,67154,CzYRFCnkbqOVb3WIMMrbfw,wife eaten la brioche least dozen time food ne...


In [54]:
pizza_business_review=pd.merge(pizza_review, pizza_business_clean_review, how='left', on='review_id')

In [55]:
pizza_business_review

,Unnamed: 0_x,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Unnamed: 0_y,clean_review
0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27,0,parking tough get get go much also get little ...
1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12,1,server extensive knowledge wine various menu s...
2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35,2,fast forward every trip nyc pitt includes visi...
3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23,3,far favorite cleveland/akron restaurant great ...
4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40,4,food good usually get italian sub pizza would ...
...,...,...,...,...,...,...,...,...,...,...,...,...
67151,67151,gcNRaJMDlSVs8YlqwpsRuw,_Rgn4SZT2wXyQnW9AD9Tog,nR3u-wQ99bop6IdGoU38KQ,1,1,0,0,I'm originally from the other side of Ohio and...,2019-09-21 00:24:03,67151,originally side ohio always really enjoyed jet...
67152,67152,yFxDOdiDb8HPaK1io81kiw,4r33dXcE1oYZxjONrhxTiA,yT1Ad3auuw1jAJEZHr7Yqw,5,1,1,1,"Yes, yes, YES!! We loved going to the place in...",2019-09-03 04:32:16,67152,yes yes yes loved going place shadyside ecstat...
67153,67153,Z_xmoLgIMB4As90t0a5eBQ,Iy6HkAzFcpeU7GF0e1FRSA,R13VkcyUIU3xG2fRFbBvBA,4,3,3,3,Went here for a lunch meeting amidst s typical...,2019-12-11 16:18:26,67153,went lunch meeting amidst typical suburban str...
67154,67154,CzYRFCnkbqOVb3WIMMrbfw,TP9Tz7gcAfK-BeT16qpZ2w,M-caugKzJ9qLOmCBfD9Nyg,1,1,0,0,My wife and I have eaten at La Brioche at leas...,2017-06-13 02:14:53,67154,wife eaten la brioche least dozen time food ne...


In [56]:
business_review=pizza_business["business_id"]

In [57]:
ID=[]
Add=[]
Keep=[]
Fix=[]
Avoid=[]
for i in business_review.tolist():
    ID.append(i)
    business_add=[]
    business_keep=[]
    business_fix=[]
    business_avoid=[]
    target_review=" ".join(pizza_business_review[pizza_business_review["business_id"]==i]["clean_review"].tolist())
    for j in Positive_Noun:
        if j in target_review:
            business_keep.append(j)
        else:
            business_add.append(j)
    for j in Negative_Noun:
        if j in target_review:
            business_fix.append(j)
        else:
            business_avoid.append(j)   
            
    
    Add.append(" ".join(business_add))
    Keep.append(" ".join(business_keep))
    Fix.append(" ".join(business_fix))
    Avoid.append(" ".join(business_avoid))
    

In [58]:
Noun_Suggestion=pd.DataFrame([ID,Add,Keep,Fix,Avoid]).transpose()

In [59]:
Noun_Suggestion=Noun_Suggestion.rename(columns={0: "business_id", 1: "Add", 2: "Keep", 3: "Fix", 4: "Avoid"})

In [60]:
Noun_Suggestion

,business_id,Add,Keep,Fix,Avoid
0,ZkzutF0P_u0C0yTulwaHkA,beer wine mozzarella bbq gelato cocktail pesto...,sausage bacon,cheese lettuce tea wing cracker,shrimp fish spaghetti caesar noodle broccoli c...
1,39lLJK_rrYY2NYomSsQdUA,wine sausage bacon mozzarella bbq gelato cockt...,beer,cheese tea wing cracker,shrimp fish spaghetti lettuce caesar noodle br...
2,0y6alZmSLnPzmG5_kP5Quw,wine bacon mozzarella bbq gelato cocktail pest...,beer sausage,cheese shrimp fish wing,spaghetti lettuce caesar noodle tea broccoli c...
3,_Kp1IPTi17wBywYOd30raA,beer wine sausage bacon mozzarella gelato cock...,bbq salami,cheese tea wing,shrimp fish spaghetti lettuce caesar noodle br...
4,crBw_Hq3gCkppirgjnIl4A,wine bacon bbq gelato cocktail pesto margherit...,beer sausage mozzarella artichoke margarita sa...,cheese spaghetti lettuce caesar wing,shrimp fish noodle tea broccoli coke patty cra...
...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,beer wine gelato cocktail margherita chocolate...,sausage bacon mozzarella bbq pesto crunchy aru...,cheese tea cracker,shrimp fish spaghetti lettuce caesar noodle br...
1556,ZZGs9idRiKRHue2RyXw9uA,beer wine sausage mozzarella bbq gelato cockta...,bacon,cheese tea,shrimp fish spaghetti lettuce caesar noodle br...
1557,v8GgrqLTs_NZZl2tkyvOgA,beer sausage bacon mozzarella bbq gelato cockt...,wine,cheese lettuce,shrimp fish spaghetti caesar noodle tea brocco...
1558,qWcaGio9JsqrvgmQiIRtIQ,beer bacon mozzarella gelato cocktail pesto ma...,wine sausage bbq crunchy artichoke cauliflower,cheese caesar broccoli wing,shrimp fish spaghetti lettuce noodle tea coke ...


Check. For example, the business_id ZkzutF0P_u0C0yTulwaHkA, its review is:

In [61]:
target_review=" ".join(pizza_business_review[pizza_business_review["business_id"]=="ZkzutF0P_u0C0yTulwaHkA"]["clean_review"].tolist())

In [62]:
"beer" in target_review

False

In [63]:
"sausage" in target_review

True

In [64]:
"cheese" in target_review

True

In [65]:
"shrimp" in target_review

False

No problem. 

# Review suggestion

In review suggestion, we will calcualte the positive review percentage of a word by business. Our group once want to set a threshold that positive percentage larger we hold that business show positive attitude toward your XXX service or word. But in the end we want to give the number and let business itself to decide customers attitude toward their service and food. 

In [66]:
Review_results

,Unnamed: 0,Word,T_Test_P_Value,T_Sig,Mean_Difference
0,1,pizza,0.000000e+00,True,1.857970
1,2,food,0.000000e+00,True,2.061958
2,3,cheese,1.174240e-81,True,1.311269
3,4,sauce,5.222103e-89,True,1.336151
4,5,service,0.000000e+00,True,2.396340
...,...,...,...,...,...
89,90,fee,2.992534e-04,True,1.090783
90,91,topping,1.772240e-15,True,1.354427
91,92,dressing,2.700788e-21,True,1.493693
92,93,olive,3.060237e-03,True,0.646953


In [67]:
Review_Word=Review_results["Word"].tolist()
Review_Word

['pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'parmesan',
 'spinach',
 'beef',
 'brunch',
 'ice',
 'chef',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gnocchi',
 'butter',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',
 'soda',
 'attitude',
 'salt',
 'seafood',
 'caesar',
 'grease',
 'cooking',
 'fee',
 'to

In [68]:
A=Review_Word
B=["review_id","business_id","stars"]
B.extend(A)
B

['review_id',
 'business_id',
 'stars',
 'pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'parmesan',
 'spinach',
 'beef',
 'brunch',
 'ice',
 'chef',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gnocchi',
 'butter',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',
 'soda',
 'attitude',
 'salt',
 'seafood',
 'caes

In [69]:
Col=B

In [70]:
Word_Review_Matrix_New=Word_Review_Matrix[Col]
Word_Review_Matrix_New

,review_id,business_id,stars,pizza,food,cheese,sauce,service,salad,restaurant,...,salt,seafood,caesar,grease,cooking,fee,topping,dressing,olive,serving
0,hfXVxrlD6fr1kXfz6sR13Q,J9f-9Prw2YVM-fiZqv2fmQ,2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDQpLLFiAxeSxWI5ipwzBQ,lpel3pRFAzlow7u2TJMz4Q,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWmT90rP7wYo57dG1cyvww,Zja7ykq3icDzkN20XX6j1w,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0jduztItUhOh3FzKGjMxZQ,hr0vxeTzlHx6fTpSSmu2AA,5,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sxfmzhMvZ8LlI1x4fhNjKg,F4kjKxXXyn3e-BYVb1mWcg,4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,gcNRaJMDlSVs8YlqwpsRuw,nR3u-wQ99bop6IdGoU38KQ,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67152,yFxDOdiDb8HPaK1io81kiw,yT1Ad3auuw1jAJEZHr7Yqw,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67153,Z_xmoLgIMB4As90t0a5eBQ,R13VkcyUIU3xG2fRFbBvBA,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67154,CzYRFCnkbqOVb3WIMMrbfw,M-caugKzJ9qLOmCBfD9Nyg,1,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
business_review=pizza_business["business_id"]

In [72]:
ID=[]
Word=[]
Business_Review_Word=[]
for i in business_review.tolist():
    ID.append(i)
    Word_Results=[]
    WordList=[]
    Target_Review=Word_Review_Matrix_New[Word_Review_Matrix_New["business_id"]==i]
    for j in Review_Word:
        results=" "
        Positive_Review_Num=Target_Review[j].tolist().count(1)
        Negative_Review_Num=Target_Review[j].tolist().count(-1)
        Neutral_Review_Num=Target_Review[j].tolist().count(0)
        Sum=Positive_Review_Num+Negative_Review_Num+Neutral_Review_Num
        # Only consider the sum of review larger than 5. 
        if Sum>5:
            results=j+" "+str(Positive_Review_Num)+" "+str(Neutral_Review_Num)+" "+str(Negative_Review_Num)
            Word_Results.append(results)
            WordList.append(j)    
    Word.append(";".join(Word_Results))
    Business_Review_Word.append(WordList)    
    

For three number following a noun, the first number is positive review number, the second number is neutral review number and third is negative review number. 

In [73]:
Word

['pizza 22 5 1;food 8 2 0;sauce 11 0 1;service 7 0 3;crust 4 2 0;garlic 4 2 1',
 'pizza 5 4 0',
 'pizza 17 2 2;sauce 6 1 0;crust 2 4 0;dough 2 4 0',
 'pizza 6 2 1',
 'pizza 43 10 0;food 11 2 1;cheese 8 4 0;sauce 13 3 0;service 9 3 3;crust 12 7 0;delivery 2 4 0;staff 7 0 1;bread 8 1 0;pepperoni 2 4 0;sausage 6 2 0;garlic 6 1 0;spaghetti 7 3 0',
 '',
 'pizza 10 2 0;food 12 4 4;cheese 3 2 2;sauce 10 4 0;service 13 2 3;salad 3 5 2;restaurant 6 5 1;menu 2 4 1;staff 4 3 0;bread 7 4 1;dinner 2 2 2;meal 4 2 1;pasta 12 2 0;sausage 4 2 1;kitchen 4 5 0',
 'pizza 31 10 6;food 13 6 3;cheese 8 9 1;service 9 2 5;crust 3 6 0;chicken 3 3 1;delivery 10 11 1;flavor 5 1 0;pepperoni 2 4 0;steak 4 2 2;hoagie 1 4 3',
 'pizza 2 5 2',
 '',
 'pizza 8 1 1',
 '',
 '',
 'pizza 10 2 0;food 11 4 0;service 7 1 0',
 '',
 'pizza 14 6 1',
 'pizza 6 3 1',
 '',
 'pizza 178 37 4;food 57 14 3;cheese 27 13 0;sauce 20 15 3;service 58 8 1;salad 25 9 1;restaurant 21 11 1;crust 48 12 2;beer 3 4 1;menu 20 12 3;staff 67 2 1;price 

In [74]:
Review_Suggestion=pd.DataFrame([ID,Word]).transpose()

In [75]:
Review_Suggestion=Review_Suggestion.rename(columns={0: "business_id", 1: "Positive_Negative_Percentage"})

In [76]:
Review_Suggestion

,business_id,Positive_Negative_Percentage
0,ZkzutF0P_u0C0yTulwaHkA,pizza 22 5 1;food 8 2 0;sauce 11 0 1;service 7...
1,39lLJK_rrYY2NYomSsQdUA,pizza 5 4 0
2,0y6alZmSLnPzmG5_kP5Quw,pizza 17 2 2;sauce 6 1 0;crust 2 4 0;dough 2 4 0
3,_Kp1IPTi17wBywYOd30raA,pizza 6 2 1
4,crBw_Hq3gCkppirgjnIl4A,pizza 43 10 0;food 11 2 1;cheese 8 4 0;sauce 1...
...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,pizza 37 6 6;food 7 1 3;service 10 4 4;salad 4...
1556,ZZGs9idRiKRHue2RyXw9uA,pizza 5 2 2
1557,v8GgrqLTs_NZZl2tkyvOgA,food 6 2 1
1558,qWcaGio9JsqrvgmQiIRtIQ,pizza 33 5 0;sauce 4 2 1;crust 11 4 1;delivery...


Check, for business_id ZkzutF0P_u0C0yTulwaHkA, count review from pizza:

In [77]:
Target_Review=Word_Review_Matrix_New[Word_Review_Matrix_New["business_id"]=="ZkzutF0P_u0C0yTulwaHkA"]

In [78]:
Target_Review["pizza"].value_counts()

 1.0    22
 0.0     5
-1.0     1
Name: pizza, dtype: int64

No problem

Find the top 3 Negative or Positive Words by business:

In [79]:
pizza_business_review=pd.read_csv("data/Split_Review.csv")
pizza_business_review.iloc[0:10]

,Unnamed: 0,Unnamed: 0.1,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Clean_Review
0,0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27,['parking is tough to get here so i do not get...
1,1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12,['the server had an extensive knowledge of win...
2,2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35,"[""fast forward every trip from nyc to pitt inc..."
3,3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23,['by far my favorite cleveland/akron restauran...
4,4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40,['all food is good here i usually get the ital...
5,5,5,Mp7qXWexqoSqKbNh7dYhJg,jAYIkCIqhDgQcp7PG8z_vA,J9f-9Prw2YVM-fiZqv2fmQ,5,1,0,0,Fries are cooked to perfection....very crispy....,2015-05-26 07:45:02,"['fries are cooked to perfection ', '', '', ''..."
6,6,6,5e_JUyKVFpYxl9qHp1MG1Q,w1LWJ4l7jDeMNmmgIUKZyg,7xHbnGo4MXFkBm6gYSu_xg,2,1,0,0,"The pizza was great, but the guys that make it...",2018-05-02 18:55:11,['the pizza was great but the guys that make i...
7,7,7,D2oYk9ipsvsYP_ls0aL9kg,9IA3P3DxrZcoAl29IYOloA,eU_713ec6fTGNO4BegRaww,4,0,0,1,"It's on Mt. Washington, but no view. The insi...",2009-08-03 20:21:18,"[""it 's on mt "", ' washington but no view ', '..."
8,8,8,CKHk7l_VriSEqPbG1PSLTw,gP0DqgbBsPTHmu_RXW8TxA,epWC7CGRbNHkXM7G2ulYzQ,3,0,0,0,There is a large dine-in area that is child fr...,2017-09-08 23:39:53,['there is a large dine-in area that is child ...
9,9,9,iFKNo8gEgKwuWTmRPmJ-MQ,SbsUVsP2gkQhJf4L2q4kjg,ZWq8Fq13JPDAqt7rPjSwcQ,5,0,0,0,"Great breakfast! Sunrise special: two eggs, a ...",2016-05-19 14:03:48,"['great breakfast ', ' sunrise special two egg..."


In [80]:
target_review=pizza_business_review[pizza_business_review["business_id"]=="J9f-9Prw2YVM-fiZqv2fmQ"]["text"]

In [81]:
Positive_Word=pd.read_csv("data/Positive_Word.csv")
Negative_Word=pd.read_csv("data/Negative_Word.csv")
Important_Positive_Word=pd.read_csv("data/Important_Positive_Word.csv")
Important_Negative_Word=pd.read_csv("data/Important_Negative_Word.csv")

In [82]:
Positive_Word_List=Positive_Word["Word"].tolist()
Negative_Word_List=Negative_Word["Word"].tolist()
Important_Positive_Word_List=Important_Positive_Word["Word"].tolist()
Important_Negative_Word_List=Important_Negative_Word["Word"].tolist()

In [83]:
replace_words={
 "n't":"not",
 "don't":"do not",
 "aren't":"are not",
 "can't":"can not",
 "couldn't":"could not",
 "didn't":"did not",
 "doesn't":"does not",
 "hadn't":"had not",
 "hasn't":"has not",
 "haven't":"have not",
 "isn't":"is not",
 "mightn't":"might not",
 "mustn't":"must not",
 "needn't":"need not",
 "shan't":"shall not",
 "shouldn't":"should not",
 "wasn't":"was not",
 "weren't":"were not",
 "won't":"will not",
 "wouldn't":"would not"}

In [84]:
Clean_Review=[]
Review=pizza_business_review["text"]
for i in range(len(Review)):
    review=Review[i]
    review=review.lower()
    review=nltk.word_tokenize(review)
    clean_review=[]
    for i in review:
        if i in set(replace_words.keys()):
            clean_review.append(replace_words.get(i))
        else:
            clean_review.append(i)
    clean_review=[i for i in clean_review if i not in '"#$%&\'()*+,-/:<=>@[\\]^_`{|}~\'\'``-']
    review=" ".join(clean_review)
    Split_Review=review.replace("!",".")
    Split_Review=Split_Review.replace("?",".")
    Split_Review=Split_Review.replace(";",".")
    Split_Review=Split_Review.split(".")
    Clean_Review.append(Split_Review)

In [85]:
r,c=pizza_business_review.shape
c=len(Review_Word)
Word_Count_Positive=[[0 for _ in range(c)] for _ in range(r)]
Word_Count_Negative=[[0 for _ in range(c)] for _ in range(r)]
Word_Count_Important_Positive=[[0 for _ in range(c)] for _ in range(r)]
Word_Count_Important_Negative=[[0 for _ in range(c)] for _ in range(r)]
for i in range(r):
    for j in range(c):
        Target=Review_Word[j]
        Review=Clean_Review[i]
        Target_Review=[]
        for Small_Review in Review:
            if Target in Small_Review.split(" "):
                Target_Review.append(Small_Review)
        Target_Review=".".join(Target_Review)
        
        if any(c.isalpha() for c in Target_Review)==True:
            Target_Review=Target_Review.split(" ")
            Word_Count_Positive[i][j]=[word for word in Positive_Word_List if word in Target_Review]
            Word_Count_Negative[i][j]=[word for word in Negative_Word_List if word in Target_Review]
            Word_Count_Important_Positive[i][j]=[word for word in Important_Positive_Word_List if word in Target_Review]
            Word_Count_Important_Negative[i][j]=[word for word in Important_Negative_Word_List if word in Target_Review]
        

These four matrix show the positive, negative, important positive and important negative word of i th review to j th noun words. For example:

In [86]:
Word_Count_Positive[0][4]

['good', 'great', 'huge', 'like', 'super']

In [87]:
Clean_Review[0]

['parking is tough to get here so i do not get to go much ',
 ' i also get a little cautious sometimes as they frequently have multiple questionable looking characters ',
 " pro 's legendary fries that used to be super huge and gave you great value for your buck good burgers if you drink it looks like they have a lot of different kinds of alcohol con 's customer service ",
 ' hahaha ',
 ' not here ',
 " if you can get more than a fart out of one of the twin 's working there you are ahead of the pack ",
 ' dirty messy tables crap everywhere ',
 ' many questionable characters here ',
 ' someone once asked me if i wanted to buy stolen jewelry off them while i was eating ',
 " i 'm being totally honest too ",
 ' their french fry portions have gotten smaller ',
 ' they used to need two baskets for a large fry and now only need one ',
 ' really expensive hot dogs ',
 ' they have strict limits on ketchup ',
 " no i 'm not joking ",
 ' 2 for large fries 1 for anything less ',
 ' i do recommend

In [88]:
Review_Word[4]

'service'

The word is "service", the small review contains service is:
    
" pro 's legendary fries that used to be super huge and gave you great value for your buck good burgers if you drink it looks like they have a lot of different kinds of alcohol con 's customer service ",
    
and the positive words is super, huge, great, good and like.

As you can see, like here is not positive words in this review, so there is no perfect way to judge review positive or negative. 


In [89]:
Word_Count_Negative[1][27]

['complaint', 'uncomfortable']

In [90]:
Review_Word[27]

'wine'

In [91]:
Clean_Review[1]

['the server had an extensive knowledge of wines their various menu selections and it was clear that we were her top priority ',
 ' the pizza and dessert were amazing ',
 ' my only complaint is that the chairs were uncomfortable i think they could add some more comfortable seating to give this more of a wine bar feel ',
 ' otherwise everything was amazing ',
 '']

No problem again. 

In [92]:
Word_Count_Important_Positive[1][27]

['comfortable']

In [93]:
Word_Count_Important_Negative[1][27]

['uncomfortable']

Both comfortable and uncomfortable occur in Word_Count_Important_Positive and Word_Count_Important_Negative word list. 

First combine the four matrix by words.

In [94]:
Review_Word

['pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'parmesan',
 'spinach',
 'beef',
 'brunch',
 'ice',
 'chef',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gnocchi',
 'butter',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',
 'soda',
 'attitude',
 'salt',
 'seafood',
 'caesar',
 'grease',
 'cooking',
 'fee',
 'to

In [95]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Positive] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
    Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
    Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(1))
    Word1.append(Word[0])
    Word2.append(Word[1])
    Word3.append(Word[2])
    Count1.append(Count[0])
    Count2.append(Count[1])
    Count3.append(Count[2])
    
Positive_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Positive_by_word=Positive_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Positive_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,good,7977,best,5354,great,5263,27586
1,food,good,5324,great,4486,delicious,1395,16216
2,cheese,good,1245,like,983,fresh,602,6100
3,sauce,good,1569,like,1104,sweet,1098,7090
4,service,great,3694,good,3067,friendly,1924,10995
...,...,...,...,...,...,...,...,...
89,fee,like,12,nice,8,good,8,70
90,topping,like,84,good,76,great,60,535
91,dressing,good,236,fresh,147,like,124,1090
92,olive,fresh,102,like,93,good,87,558


This table show you the top 3 positive words towards a target word and their count, and the sum of all positive reviews. 

Repeat with other three

In [96]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Negative] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
    Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
    Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(-1))
    Word1.append(Word[0])
    Word2.append(Word[1])
    Word3.append(Word[2])
    Count1.append(Count[0])
    Count2.append(Count[1])
    Count3.append(Count[2])
    
Negative_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Negative_by_word=Negative_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Negative_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,bad,795,wrong,467,worst,451,2564
1,food,bad,553,terrible,246,wrong,221,2063
2,cheese,bad,96,wrong,56,disappointed,45,624
3,sauce,bad,109,wrong,65,die,57,663
4,service,terrible,602,bad,588,poor,485,2592
...,...,...,...,...,...,...,...,...
89,fee,charged,24,bad,2,horrible,2,31
90,topping,missing,15,wrong,11,charged,11,97
91,dressing,wrong,18,missing,14,forget,12,128
92,olive,crushed,9,bad,6,die,4,45


In [97]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Important_Positive] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
    Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
    Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(1))
    Word1.append(Word[0])
    Word2.append(Word[1])
    Word3.append(Word[2])
    Count1.append(Count[0])
    Count2.append(Count[1])
    Count3.append(Count[2])
    
Important_Positive_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Important_Positive_by_word=Important_Positive_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Important_Positive_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,delicious,2604,fresh,1532,free,671,27586
1,food,delicious,1395,fresh,623,cheap,213,16216
2,cheese,fresh,602,delicious,525,cheap,85,6100
3,sauce,fresh,676,delicious,637,cheap,53,7090
4,service,delicious,379,clean,185,fresh,148,10995
...,...,...,...,...,...,...,...,...
89,fee,cheap,7,free,3,efficient,1,70
90,topping,fresh,58,delicious,30,free,19,535
91,dressing,fresh,147,delicious,100,healthy,12,1090
92,olive,fresh,102,delicious,42,free,8,558


In [98]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Important_Negative] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    if B!=[]:
        Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
        Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
        Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(-1))
        Word1.append(Word[0])
        Count1.append(Count[0])
        if len(Word)>1:
            Word2.append(Word[1])
            Count2.append(Count[1])
        else:
            Word2.append(" ")
            Count2.append(0)
        if len(Word)>2:
            Word3.append(Word[2])
            Count3.append(Count[2])
        else:
            Word3.append(" ")
            Count3.append(0)
    else:
            Word1.append(" ")
            Count1.append(0)
            Word2.append(" ")
            Count2.append(0)
            Word3.append(" ")
            Count3.append(0)
            Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(-1))
Important_Negative_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Important_Negative_by_word=Important_Negative_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Important_Negative_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,rude,140,snob,102,sick,56,2564
1,food,rude,128,sick,47,starving,24,2063
2,cheese,stingy,20,boring,14,rude,8,624
3,sauce,boring,19,stingy,18,lazy,5,663
4,service,rude,192,unprofessional,19,shitty,19,2592
...,...,...,...,...,...,...,...,...
89,fee,outrageous,1,shitty,1,inconvenient,1,31
90,topping,rude,3,outrageous,2,sick,1,97
91,dressing,boring,2,insulting,1,rude,1,128
92,olive,snob,1,delay,1,uncomfortable,1,45


In [99]:
Important_Negative_by_word.to_csv("data/Important_Negative_by_word.csv")
Important_Positive_by_word.to_csv("data/Important_Positive_by_word.csv")
Negative_by_word.to_csv("data/Important_Negative_by_word.csv")
Positive_by_word.to_csv("data/Important_Positive_by_word.csv")

No problem

Find other information we want in suggestion in Shiny app. 

In [100]:
ID=[]
Review_Num=[]
Average_Review_Star_Rating=[]
for i in business_review.tolist():
    ID.append(i)
    Review_Num.append(len(pizza_review[pizza_review["business_id"]==i]["stars"]))
    Average_Review_Star_Rating.append(round(pizza_review[pizza_review["business_id"]==i]["stars"].mean(),3))

In [101]:
Review_NUM=pd.DataFrame([ID,Review_Num,Average_Review_Star_Rating]).transpose()
Review_NUM=Review_NUM.rename(columns={0: "business_id", 1: "Review_Num",2: "Average_Review_Star_Rating"})
Review_NUM

,business_id,Review_Num,Average_Review_Star_Rating
0,ZkzutF0P_u0C0yTulwaHkA,32,3.875
1,39lLJK_rrYY2NYomSsQdUA,11,2.455
2,0y6alZmSLnPzmG5_kP5Quw,21,4.524
3,_Kp1IPTi17wBywYOd30raA,15,2.267
4,crBw_Hq3gCkppirgjnIl4A,63,4.429
...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,60,3.7
1556,ZZGs9idRiKRHue2RyXw9uA,10,3
1557,v8GgrqLTs_NZZl2tkyvOgA,12,2.75
1558,qWcaGio9JsqrvgmQiIRtIQ,39,4.231


In [102]:
Other_Information=pizza_business[["business_id","name",'address',"city",'latitude','longitude','stars']]
Other_Information

,business_id,name,address,city,latitude,longitude,stars
0,ZkzutF0P_u0C0yTulwaHkA,Lelulos Pizzeria,311 Unity Center Rd,Pittsburgh,40.489996,-79.779288,4.0
1,39lLJK_rrYY2NYomSsQdUA,Marco's Pizza,24335 Chagrin Blvd,Cleveland,41.465789,-81.506349,2.5
2,0y6alZmSLnPzmG5_kP5Quw,J J's Pizza,20542 Lorain Rd,Cleveland,41.448341,-81.847644,4.5
3,_Kp1IPTi17wBywYOd30raA,Pizza Hut,8609 University Blvd,Pittsburgh,40.507268,-80.222632,2.0
4,crBw_Hq3gCkppirgjnIl4A,Famous Yeti's Pizza,971 N Page St,Madison,42.926526,-89.222592,4.5
...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,Blaze Fast-Fire'd Pizza,2610 N Prospect Ave,Urbana-Champaign,40.147209,-88.254940,3.5
1556,ZZGs9idRiKRHue2RyXw9uA,Marco's Pizza,5877 Broadview Rd,Cleveland,41.403398,-81.689779,3.0
1557,v8GgrqLTs_NZZl2tkyvOgA,Cleveland Clinic Cafeteria,"9500 Euclid Ave, Bldg C, Fl 1",Cleveland,41.501753,-81.621448,3.0
1558,qWcaGio9JsqrvgmQiIRtIQ,Country Style Pizza,536 Dewey Ave,Pittsburgh,40.356300,-80.110600,4.0


In [103]:
Suggestion=pd.merge(Business_Suggestion,Noun_Suggestion,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Review_Suggestion,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Other_Information,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Review_NUM,how='left', on='business_id')
Suggestion

,business_id,Alcohol,BikeParking,GoodForKids,NoiseLevel,RestaurantsGoodForGroups,RestaurantsReservations,WiFi,RestaurantsDelivery,BusinessParking,...,Avoid,Positive_Negative_Percentage,name,address,city,latitude,longitude,stars,Review_Num,Average_Review_Star_Rating
0,ZkzutF0P_u0C0yTulwaHkA,0,1,1,1,1,1,0,0,0,...,shrimp fish spaghetti caesar noodle broccoli c...,pizza 22 5 1;food 8 2 0;sauce 11 0 1;service 7...,Lelulos Pizzeria,311 Unity Center Rd,Pittsburgh,40.489996,-79.779288,4.0,32,3.875
1,39lLJK_rrYY2NYomSsQdUA,0,0,1,1,0,0,0,0,0,...,shrimp fish spaghetti lettuce caesar noodle br...,pizza 5 4 0,Marco's Pizza,24335 Chagrin Blvd,Cleveland,41.465789,-81.506349,2.5,11,2.455
2,0y6alZmSLnPzmG5_kP5Quw,0,1,1,1,1,0,0,0,0,...,spaghetti lettuce caesar noodle tea broccoli c...,pizza 17 2 2;sauce 6 1 0;crust 2 4 0;dough 2 4 0,J J's Pizza,20542 Lorain Rd,Cleveland,41.448341,-81.847644,4.5,21,4.524
3,_Kp1IPTi17wBywYOd30raA,1,0,1,1,1,0,0,0,1,...,shrimp fish spaghetti lettuce caesar noodle br...,pizza 6 2 1,Pizza Hut,8609 University Blvd,Pittsburgh,40.507268,-80.222632,2.0,15,2.267
4,crBw_Hq3gCkppirgjnIl4A,1,1,1,1,1,0,1,0,1,...,shrimp fish noodle tea broccoli coke patty cra...,pizza 43 10 0;food 11 2 1;cheese 8 4 0;sauce 1...,Famous Yeti's Pizza,971 N Page St,Madison,42.926526,-89.222592,4.5,63,4.429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,0,0,1,0,1,0,0,0,0,...,shrimp fish spaghetti lettuce caesar noodle br...,pizza 37 6 6;food 7 1 3;service 10 4 4;salad 4...,Blaze Fast-Fire'd Pizza,2610 N Prospect Ave,Urbana-Champaign,40.147209,-88.254940,3.5,60,3.7
1556,ZZGs9idRiKRHue2RyXw9uA,0,0,1,1,1,0,0,0,0,...,shrimp fish spaghetti lettuce caesar noodle br...,pizza 5 2 2,Marco's Pizza,5877 Broadview Rd,Cleveland,41.403398,-81.689779,3.0,10,3
1557,v8GgrqLTs_NZZl2tkyvOgA,0,0,1,0,1,0,1,1,0,...,shrimp fish spaghetti caesar noodle tea brocco...,food 6 2 1,Cleveland Clinic Cafeteria,"9500 Euclid Ave, Bldg C, Fl 1",Cleveland,41.501753,-81.621448,3.0,12,2.75
1558,qWcaGio9JsqrvgmQiIRtIQ,0,1,1,1,1,0,0,0,1,...,shrimp fish spaghetti lettuce noodle tea coke ...,pizza 33 5 0;sauce 4 2 1;crust 11 4 1;delivery...,Country Style Pizza,536 Dewey Ave,Pittsburgh,40.356300,-80.110600,4.0,39,4.231


In [104]:
Suggestion.to_csv("data/Suggestion.csv")